<a href="https://colab.research.google.com/github/jonathancagua/stock-trading-DQL/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget https://raw.githubusercontent.com/jonathancagua/stock-trading-DQL/main/YNDX_150101_151231.csv

import pandas as pd

data = pd.read_csv("YNDX_150101_151231.csv")
data.head()

--2025-06-08 23:13:00--  https://raw.githubusercontent.com/jonathancagua/stock-trading-DQL/main/YNDX_150101_151231.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9009713 (8.6M) [text/plain]
Saving to: ‘YNDX_150101_151231.csv’

YNDX_150101_151231. 100%[===================>]   8.59M  --.-KB/s    in 0.06s   

2025-06-08 23:13:00 (136 MB/s) - ‘YNDX_150101_151231.csv’ saved [9009713/9009713]



,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,20150105,100100,1064.0,1064.0,1064.0,1064.0,0
1,20150105,100200,1064.0,1064.0,1064.0,1064.0,0
2,20150105,100300,1064.0,1064.0,1064.0,1064.0,0
3,20150105,100400,1064.0,1064.0,1064.0,1064.0,0
4,20150105,100500,1064.0,1064.0,1064.0,1064.0,0


In [4]:
columns={"<DATE>":"date" ,"<TIME>":"time" ,
         "<OPEN>":"open" ,"<HIGH>":"high" ,
         "<LOW>":"low" ,"<CLOSE>":"close" ,
         "<VOL>":"vol"}
data = data.rename(columns=columns)

In [5]:
data.head()

,date,time,open,high,low,close,vol
0,20150105,100100,1064.0,1064.0,1064.0,1064.0,0
1,20150105,100200,1064.0,1064.0,1064.0,1064.0,0
2,20150105,100300,1064.0,1064.0,1064.0,1064.0,0
3,20150105,100400,1064.0,1064.0,1064.0,1064.0,0
4,20150105,100500,1064.0,1064.0,1064.0,1064.0,0


In [6]:
del data["vol"] ,data["date"] ,data["time"]

In [7]:
data["remove"] = data.apply(lambda x: all([abs(i - x[0]) < 1e-8 for i in x]) ,axis=1)

<ipython-input-7-8645988fc25a>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data["remove"] = data.apply(lambda x: all([abs(i - x[0]) < 1e-8 for i in x]) ,axis=1)


In [8]:
data = data.query("remove == False").reset_index(drop=True)

In [9]:
del data["remove"]

In [10]:
data["high"] = (data["high"] - data["open"]) / data["open"]
data["low"]  = (data["low"] - data["open"]) / data["open"]
data["close"] = (data["close"] - data["open"]) / data["open"]

In [11]:
import numpy as np

In [12]:
class YNDX_env:
    def __init__(self ,data ,obs_bars = 10 ,test = False ,commission_perc=0.1):
        self.data = data
        self.obs_bars = obs_bars
        self.have_position = False
        self.open_price = 0
        self.test = test
        self.commission_perc = commission_perc
        if test == False:
            self.curr_step =  np.random.choice(self.data.high.shape[0] - self.obs_bars*10) + self.obs_bars
        else:
            self.curr_step = self.obs_bars

        self.state = self.data[self.curr_step - self.obs_bars : self.curr_step]



    def step(self ,action):
        reward = 0
        done = False
        relative_close = self.state["close"][self.curr_step - 1]
        open = self.state["open"][self.curr_step - 1]
        close = open * ( 1 + relative_close)

        if action == "buy" and self.have_position == False:
            self.have_position = True
            self.open_price = close
            reward -= self.commission_perc

        elif action == "close" and self.have_position == True:
            reward -= self.commission_perc
            if self.test == False:
                done = True

            reward += 100.0 * (close -  self.open_price) / self.open_price

            self.have_position = False
            self.open_price = 0.0

        self.curr_step = self.curr_step + 1
        self.state = self.data[self.curr_step - self.obs_bars : self.curr_step]

        if self.curr_step == len(self.data) - 1:
            done = True

        state = np.zeros((5 ,self.obs_bars), dtype=np.float32)
        state[0] = self.state.high.to_list()
        state[1] = self.state.low.to_list()
        state[2] = self.state.close.to_list()
        state[3] = int(self.have_position)
        if self.have_position:
            state[4] =  (close -  self.open_price) / self.open_price
        return state ,reward , done


In [13]:
actions = {0:"do_nothing" ,1:"buy" ,2:"close"}

In [14]:
YNDX = YNDX_env(data,test=False,obs_bars=50)

In [15]:
state ,reward ,done = YNDX.step("do_nothing")

In [16]:
state.shape

(5, 50)

In [17]:
input_shape = state.shape

In [18]:
import torch

In [19]:
class dueling_Conv1D_Q_Net(torch.nn.Module):
    def __init__(self, input_depth_length , output_shape):
        super(dueling_Conv1D_Q_Net, self).__init__()
        self.conv1 = torch.nn.Conv1d(input_depth_length, 128, 5)
        self.conv2 = torch.nn.Conv1d(128, 128, 5)
        self.state_value_linear1 =  torch.nn.Linear(5376 , 512)
        self.state_value_linear2 =  torch.nn.Linear(512, 1)
        self.advantage_linear1 =  torch.nn.Linear(5376 ,512)
        self.advantage_linear2 =  torch.nn.Linear(512 ,output_shape)
        self.activation = torch.nn.ReLU()
        self.flatten = torch.nn.Flatten()
    def forward(self ,x):
        x = self.conv1(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.flatten(x)
        x_state_val = self.state_value_linear1(x)
        x_state_val = self.activation(x_state_val)
        x_state_val = self.state_value_linear2(x_state_val)
        x_advantage = self.advantage_linear1(x)
        x_advantage = self.activation(x_advantage)
        x_advantage = self.advantage_linear2(x_advantage)
        y = x_state_val + x_advantage - x_advantage.mean(dim=1).unsqueeze(1)
        return y

In [20]:
def get_batch_for_nsteps_dqn(replay ,batch_size ,nsteps = 1 ,device = "cpu"):

    if nsteps < 1:
        nsteps = 1

    minibatch_idx = random.sample(range(len(replay)), batch_size)
    minibatch = [replay[idx] for idx in minibatch_idx]
    nsteps_next_state_batch = []
    nsteps_reward_batch = []
    nsteps_done_batch = []

    state1_batch = torch.cat([s1 for (s1 ,a ,r ,s2 ,d) in minibatch]).float().to(device)
    action1_batch = torch.tensor([a for (s1 ,a ,r ,s2 ,d) in minibatch]).long().to(device)

    for exp_idx in minibatch_idx:
        nsteps_next_state = []
        nsteps_reward = []

        for step in range(nsteps):
            exp = replay[exp_idx + step]
            s1 ,a ,r ,s2 ,d = exp
            nsteps_reward.append(r)

            if d == True or step == nsteps - 1 or exp_idx + step == len(replay) - 1 :
                nsteps_next_state_batch.append(s2)
                nsteps_done_batch.append(d)
                break

        nsteps_reward_batch.append(nsteps_reward)

    nsteps_next_state_batch = torch.cat([s for s in nsteps_next_state_batch]).float().to(device)
    nsteps_done_batch = torch.tensor(nsteps_done_batch).long().to(device)

    return state1_batch ,action1_batch ,nsteps_next_state_batch ,nsteps_reward_batch  ,nsteps_done_batch

In [21]:
def batch_target_for_nsteps_dqn(nsteps_reward_batch  ,gamma ,maxQ ,nsteps_done_batch ,device="cpu"):
    Y_list = []
    for i , nsteps_reward in enumerate(nsteps_reward_batch):
        gamma_list = [gamma**i for i in range(len(nsteps_reward))]
        gamma_array = np.array(gamma_list)
        nsteps_reward = np.array(nsteps_reward)
        Y = torch.tensor(nsteps_reward * gamma_list).sum().to(device) + gamma**(len(nsteps_reward)) * maxQ[i] * (1 - nsteps_done_batch[i])
        Y_list.append(Y)
    return torch.tensor([Y for Y in Y_list]).float().to(device)

In [22]:
import random


In [23]:
from IPython.display import clear_output

In [24]:
def preprocess_state(state ,input_shape , add_noise = False):
    if add_noise == True:
        state = state.reshape(*input_shape)  + np.random.rand(*input_shape)/100.0
    else:
        state = state.reshape(*input_shape)
    return state

In [25]:
def get_action(Q_val ,num_actions ,epsilon):
    if (random.random() < epsilon):
        action = np.random.randint(0,num_actions)
    else:
        action = np.argmax(Q_val ,1)[0]
    return action

In [26]:
def update(loss , optimizer):
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
from collections import deque
import copy
memory_size = 100000
batch_size = 32
replay = deque(maxlen=memory_size)
gamma = 0.99
learning_rate =  0.0001
sync_freq = 1000
device="cuda"
output_shape = len(actions)

Agent_NN = dueling_Conv1D_Q_Net(input_shape[0] ,output_shape).to(device)
target_NN = copy.deepcopy(Agent_NN)
target_NN.load_state_dict(Agent_NN.state_dict())

optimizer = torch.optim.RMSprop(Agent_NN.parameters(), lr=learning_rate)
loss_fn = torch.nn.MSELoss()
all_rewards_list = []
mean_rewards = []
Q_losses = []
epsilon = 1.
k = 0
while(k < 2600000):
    game = YNDX_env(data ,commission_perc=0.1 ,obs_bars=50)
    state1 ,_ ,_  = game.step("do_nothing")
    state1 = np.array(state1)
    state1 = preprocess_state(state1 ,(1,*input_shape) ,add_noise = False)
    state1 = torch.from_numpy(state1).float()
    status = 1
    episode_rewards = []
    while(status == 1):
        k = k + 1
        with torch.no_grad():
            qval = Agent_NN(state1.to(device))
            qval_ = qval.data.cpu().numpy()
        action = get_action(qval_ , 3, epsilon)
        action_name = actions[action]
        state2 ,reward ,done = game.step(action_name)
        state2 = np.array(state2)
        state2 = preprocess_state(state2 ,(1,*input_shape) ,add_noise = False)
        state2 = torch.from_numpy(state2).float()
        exp =  (state1, action, reward, state2, done)
        episode_rewards.append(reward)
        replay.append(exp)


        if len(replay) >= 10000:
            state1_batch ,action1_batch ,nsteps_next_state_batch ,nsteps_reward_batch ,nsteps_done_batch = get_batch_for_nsteps_dqn(replay =replay,
                                                                                                                                    batch_size = batch_size,
                                                                                                                                    nsteps=2 ,device=device)


            state_batch = torch.cat([state1_batch ,nsteps_next_state_batch],dim=0)
            Q_1_and_n = Agent_NN(state_batch)
            Q1 = Q_1_and_n[:batch_size]

            with torch.no_grad():
                Q_next_state = Q_1_and_n[batch_size:]
                selected_nodes_for_target_network =  Q_next_state.max(1)[1]
                Q_next_state = target_NN(nsteps_next_state_batch)

            best_Q_next_state = Q_next_state.gather(dim=1,index=selected_nodes_for_target_network.unsqueeze(dim=1)).squeeze()
            Y = batch_target_for_nsteps_dqn(nsteps_reward_batch  ,gamma ,best_Q_next_state ,nsteps_done_batch ,device=device)
            X = Q1.gather(dim=1,index=action1_batch.unsqueeze(dim=1)).squeeze()
            loss = loss_fn(X, Y)
            #loss = losses.mean()
            print("current step" ,k)
            print("mean_reward:" ,mean_rewards[-1])
            print(loss.item())
            clear_output(wait=True)
            Q_losses.append(loss.item())
            update(loss ,optimizer)


        state1 = state2
        if k % sync_freq == 0:
            target_NN.load_state_dict(Agent_NN.state_dict())


        if done == True:
            mean_rewards.append(np.array(episode_rewards).sum())
            status = 0

    if k < 1000000:
        epsilon = 1 - 0.9 * (k / 1000000)
    else:
        epsilon = 0.1

In [55]:
!wget https://raw.githubusercontent.com/jonathancagua/stock-trading-DQL/main/YNDX_160101_161231.csv

--2025-06-08 23:38:39--  https://raw.githubusercontent.com/jonathancagua/stock-trading-DQL/main/YNDX_160101_161231.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9369705 (8.9M) [text/plain]
Saving to: ‘YNDX_160101_161231.csv’

YNDX_160101_161231. 100%[===================>]   8.94M  --.-KB/s    in 0.08s   

2025-06-08 23:38:40 (115 MB/s) - ‘YNDX_160101_161231.csv’ saved [9369705/9369705]



In [56]:
test = pd.read_csv("YNDX_160101_161231.csv")


In [57]:
test.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,20160104,100100,1148.9,1148.9,1148.9,1148.9,0
1,20160104,100200,1148.9,1148.9,1148.9,1148.9,50
2,20160104,100300,1149.0,1149.0,1149.0,1149.0,33
3,20160104,100400,1149.0,1149.0,1149.0,1149.0,4
4,20160104,100500,1153.0,1153.0,1153.0,1153.0,0


In [58]:
columns={"<DATE>":"date" ,"<TIME>":"time" ,
         "<OPEN>":"open" ,"<HIGH>":"high" ,
         "<LOW>":"low" ,"<CLOSE>":"close" ,
         "<VOL>":"vol"}
test = test.rename(columns=columns)

In [59]:
test.head()

,date,time,open,high,low,close,vol
0,20160104,100100,1148.9,1148.9,1148.9,1148.9,0
1,20160104,100200,1148.9,1148.9,1148.9,1148.9,50
2,20160104,100300,1149.0,1149.0,1149.0,1149.0,33
3,20160104,100400,1149.0,1149.0,1149.0,1149.0,4
4,20160104,100500,1153.0,1153.0,1153.0,1153.0,0


In [60]:
del test["vol"] ,test["date"] ,test["time"]

In [61]:
test["remove"] = test.apply(lambda x: all([abs(i - x[0]) < 1e-8 for i in x]) ,axis=1)

<ipython-input-61-910a8ba581e1>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test["remove"] = test.apply(lambda x: all([abs(i - x[0]) < 1e-8 for i in x]) ,axis=1)


In [62]:
test = test.query("remove == False").reset_index(drop=True)

In [63]:
del test["remove"]

In [64]:
test["high"] = (test["high"] - test["open"]) / test["open"]
test["low"]  = (test["low"] - test["open"]) / test["open"]
test["close"] = (test["close"] - test["open"]) / test["open"]

In [ ]:
game = YNDX_env(test ,commission_perc=0.1 ,obs_bars=50 ,test=True)
state1 ,_ ,_  = game.step("do_nothing")
state1 = np.array(state1)
state1 = preprocess_state(state1 ,(1,*input_shape) ,add_noise = False)
state1 = torch.from_numpy(state1).float()
status = 1
rewards = []
i = 0
while(status == 1):
        with torch.no_grad():
            qval = Agent_NN(state1.to(device))
            qval_ = qval.data.cpu().numpy()
        action = np.argmax(qval_ ,1)[0]
        action_name = actions[action]
        print(action_name)
        i = i + 1
        print(i)
        state2 ,reward ,done  = game.step(action_name)
        rewards.append(reward)
        state2 = np.array(state2)
        state2 = preprocess_state(state2 ,(1,*input_shape) ,add_noise = False)
        state2 = torch.from_numpy(state2).float()
        state1 = state2
        clear_output(wait=True)
        if done == True:
            status = 0

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(np.cumsum(rewards))
plt.xlabel("steps",fontsize=22)
plt.ylabel("rewards",fontsize=22)